In [7]:
import h5py
import numpy as np
# import matplotlib.pyplot as plt
from tifffile import imwrite, imread, tiffcomment, TiffFile
from ome_types import from_tiff
from glob import glob
import os
import affine as affine
from scipy.ndimage import affine_transform
from timeit import default_timer as timer
from npy2bdv import BdvWriter
from napari import Viewer

In [2]:
## define affine matrices for deskewing

def deskew(opm_angle, size_x, size_z):
    matrix_1_iso = affine.scale_matrix(sz=size_z/size_x)  # scale like in MVR calibration
    matrix_2_rot = affine.rotation_matrix(theta=np.pi/2, axis='x')  # make so stack direction is y not z(prime) like in lab
    matrix_3_scale = affine.scale_matrix(sz=np.sin(opm_angle))  # scale before shear to preserve aspect ratio
    matrix_4_shear = affine.skew_matrix(opm_angle)  # shear to deskew

    total_matrix = matrix_4_shear @ matrix_3_scale @ matrix_2_rot @ matrix_1_iso
    return total_matrix

def just_rotate_90():
    matrix_2_rot = affine.rotation_matrix(theta=np.pi/2, axis='x')  # make so stack direction is y not z(prime) like in lab
    return matrix_2_rot

In [3]:
root_dir = "E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0/dOPM_t0000_p0000_z0000_c0000_view1/HamamatsuHam_DCAM-dopm/"
stack_path = os.path.join(root_dir, "HamamatsuHam_DCAM-dopm_MMStack.ome.tif")


omexml = from_tiff(stack_path)
iminfo = omexml.images[0]

size_x = iminfo.pixels.size_x
size_y = iminfo.pixels.size_y
size_z = len(omexml.images)
vox_size_x_um = iminfo.pixels.physical_size_x
vox_size_y_um = iminfo.pixels.physical_size_y
vox_size_z_um = iminfo.pixels.physical_size_z

print("size_x", size_x)
print("size_y", size_y)
print("size_z", size_z)

imstack = np.zeros((size_x, size_y, 403), dtype=np.uint16)

with TiffFile(stack_path) as tif:
    print(tif.pages)
    for n, page in enumerate(tif.pages):
        # print("page", n)
        imstack[:,:,n] = page.asarray().T # transpose for x->y y->x



size_x 2304
size_y 2304
size_z 1600
<tifffile.TiffPages @9950>


In [22]:
%%time

root_dir = "E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0/dOPM_t0000_p0000_z0000_c0000_view1/HamamatsuHam_DCAM-dopm/"
stack_path = os.path.join(root_dir, "HamamatsuHam_DCAM-dopm_MMStack.ome.tif")

nz, ny, nx = 403, 2304, 2304

bdv_savedir = os.path.join(root_dir, "bdvtest")
os.makedirs(bdv_savedir, exist_ok=True)

fname = os.path.join(bdv_savedir, "stack.h5")
bdv_writer = BdvWriter(fname, nchannels=1, nilluminations=1, nangles=1)

# bdv_writer.set_attribute_labels('channel', ('488', '561'))
# bdv_writer.set_attribute_labels('illumination', ('L', 'R'))
bdv_writer.set_attribute_labels('angle', ('0'))

imstack = np.zeros((size_x, size_y, 403), dtype=np.uint16)

omexml = from_tiff(stack_path)
iminfo = omexml.images[0]

size_x = iminfo.pixels.size_x
size_y = iminfo.pixels.size_y
size_z = len(omexml.images)
vox_size_x_um = iminfo.pixels.physical_size_x
vox_size_y_um = iminfo.pixels.physical_size_y
vox_size_z_um = iminfo.pixels.physical_size_z

with TiffFile(stack_path) as tif:
    print(tif.pages)
    for n, page in enumerate(tif.pages):
        # print("page", n)
        imstack[:,:,n] = page.asarray().T # transpose for x->y y->x

affine_matrix = just_rotate_90(square=False)
OPM_ANGLE = 45
angles = [0, 90]
for a in range(1):
    bdv_writer.append_view(imstack, angle=angles[a],
            calibration=(vox_size_x_um, vox_size_y_um, vox_size_z_um))
    bdv_writer.write_xml()
    bdv_writer.append_affine(
        affine.rotation_matrix(theta=np.pi/2, axis='x'),
        name_affine='90 deg rotation about x')
    skew_angle = np.pi*(angles[a]-OPM_ANGLE)/180
    bdv_writer.append_affine(
        affine.scale_matrix(sz=np.sin(skew_angle),
        name_affine='scale before shear'))
    bdv_writer.append_affine(
        affine.skew_matrix(skew_angle),
        name_affine='shear to deskew')
bdv_writer.write_xml()
bdv_writer.close()
print(f"dataset in {fname}")



<tifffile.TiffPages @9950>
dataset in E:/dopm_reslicing/y_stage_scan_20241216-171205_c0p0/dOPM_t0000_p0000_z0000_c0000_view1/HamamatsuHam_DCAM-dopm/bdvtest\stack.h5
CPU times: total: 53.6 s
Wall time: 1min


In [5]:


just_rot = just_rotate_90()
imstack_rot = affine_transform(imstack, just_rot)

In [13]:
# make imstack_rot (z,y,x) instead of (x,y,z)
imstack_rot2 = np.moveaxis(imstack_rot, [0,1,2], [2,1,0])
imstack_rot2.shape

imwrite("testrot.tif", imstack_rot2)

In [10]:
viewer = Viewer()
viewer.add_image(imstack_rot)
viewer.show()

ImportError: An error occured when importing Qt dependencies.  Cannot show napari window.  See cause above